In [27]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz
pd.reset_option('display.max_rows')
#pd.set_option('display.max_rows', None)

In [28]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("TEAM_NAME_base", group_keys=False).apply(lambda x: shift_col(x, col_name))

In [29]:
folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/nba_api/data/teams_stats/2024_04_01.csv"
df = pd.read_csv(folder_path, index_col=0)
df = df.reset_index(drop=True)

In [30]:
df['date_next'] = add_col(df, 'Date')
df['date_next'].fillna('2024-04-02', inplace=True)
dates = df['date_next'].unique()

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_86963/2200988528.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("TEAM_NAME_base", group_keys=False).apply(lambda x: shift_col(x, col_name))
/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_86963/2958482147.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace

In [31]:
mapping = {'Brooklyn Nets': 'BRK', 'Golden State Warriors': 'GSW', 'Los Angeles Lakers': 'LAL',
       'Milwaukee Bucks': 'MIL', 'Boston Celtics': 'BOS', 'Charlotte Hornets': 'CHO',
       'Chicago Bulls': 'CHI', 'Cleveland Cavaliers': 'CLE', 'Denver Nuggets': 'DEN',
       'Detroit Pistons': 'DET', 'Houston Rockets': 'HOU', 'Indiana Pacers': 'IND',
       'Memphis Grizzlies': 'MEM', 'Minnesota Timberwolves': 'MIN',
       'New Orleans Pelicans': 'NOP', 'New York Knicks': 'NYK', 'Oklahoma City Thunder': 'OKC',
       'Orlando Magic': 'ORL', 'Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHO',
       'Portland Trail Blazers': 'POR', 'Sacramento Kings': 'SAC', 'San Antonio Spurs': 'SAS',
       'Toronto Raptors': 'TOR', 'Utah Jazz': 'UTA', 'Washington Wizards': 'WAS',
       'Atlanta Hawks': 'ATL', 'Dallas Mavericks': 'DAL', 'Los Angeles Clippers': 'LAC', 'Miami Heat': 'MIA'}

In [32]:
# Define time zones
dates_utc = []
est_timezone = pytz.timezone('America/New_York')  # Eastern Standard Time (EST)
utc_timezone = pytz.timezone('UTC')

threshold_value = pd.to_datetime('2020-06-27T03:55:00Z')

full = []
for date in dates:
    date = date[:10]
    # Create a datetime object for the given time in EST
    est_time = datetime.strptime(f'{date} 10:00 AM', '%Y-%m-%d %I:%M %p')
    est_time = est_timezone.localize(est_time)

    # Convert EST time to UTC
    utc_time = est_time.astimezone(utc_timezone)
    if utc_time >= threshold_value:
        final_time = utc_time.strftime('%Y-%m-%dT%H:%M:%SZ')
        dates_utc.append(final_time)

In [33]:
KEY = 'dc34ee156925bf51d4a4c33c87d440fc'
SPORT = 'basketball_nba'
REGION = 'us'
MARKET = 'h2h'
FORMAT = 'decimal'
full = []
# adjust number of dates 
DATES = dates_utc
for DATE in DATES:
    response = requests.get(f'https://api.the-odds-api.com/v4/historical/sports/{SPORT}/odds',
        params={
            'api_key': KEY,
            'regions': REGION,
            'markets': MARKET,
            'oddsFormat': FORMAT,
            'date': DATE,
        }
    ).json()
    for i in range(len(response['data'])):
        # [Timestamp, good team, evil team, fanduel odds good team wins, draftkings odds, caesars odds]
        #best_odds = [[None,None,None,None,float('-inf')] for _ in range(2)]
        bookmakers = response['data'][i]['bookmakers']
        for index, data in enumerate(bookmakers):
            best_odds = [[None,None,None,None,float('-inf')] for _ in range(2)]
            if response['data'][i]['bookmakers'][index]['title'] in ["FanDuel", "BetMGM", "DraftKings", ""]:
                for outcome in range(2):
                    odds = response['data'][i]['bookmakers'][index]['markets'][0]['outcomes'][outcome]['price']
                    current_odds = best_odds[outcome][4]
                    best_odds[outcome][0] = response['timestamp']
                    best_odds[outcome][1] = response['data'][i]['id']
                    best_odds[outcome][2] = response['data'][i]['bookmakers'][index]['title']
                    best_odds[outcome][3] = response['data'][i]['bookmakers'][index]['markets'][0]['outcomes'][outcome]['name']
                    best_odds[outcome][4] = odds
                full.extend(best_odds)

In [34]:
timestamp = []
Id = []
sportsbook = []
team = []
odds = []
for i in range(len(full)):
    timestamp.append(full[i][0])
    Id.append(full[i][1])
    sportsbook.append(full[i][2])
    team.append(full[i][3])
    odds.append(full[i][4])
df = pd.DataFrame({"Timestamp": timestamp,"Id": Id, "Sportsbook": sportsbook, "Team": team, "Odds": odds})

In [35]:
df

,Timestamp,Id,Sportsbook,Team,Odds
0,2024-03-23T13:55:39Z,16eb396a044d9dec5d240604df0c13bf,DraftKings,Brooklyn Nets,3.40
1,2024-03-23T13:55:39Z,16eb396a044d9dec5d240604df0c13bf,DraftKings,New York Knicks,1.34
2,2024-03-23T13:55:39Z,16eb396a044d9dec5d240604df0c13bf,FanDuel,Brooklyn Nets,3.40
3,2024-03-23T13:55:39Z,16eb396a044d9dec5d240604df0c13bf,FanDuel,New York Knicks,1.34
4,2024-03-23T13:55:39Z,16eb396a044d9dec5d240604df0c13bf,BetMGM,Brooklyn Nets,3.40
...,...,...,...,...,...
481,2024-04-02T13:55:39Z,3d41daf5b30be266bf64bf38ca1da2bb,DraftKings,Sacramento Kings,2.14
482,2024-04-02T13:55:39Z,3d41daf5b30be266bf64bf38ca1da2bb,FanDuel,Los Angeles Clippers,1.74
483,2024-04-02T13:55:39Z,3d41daf5b30be266bf64bf38ca1da2bb,FanDuel,Sacramento Kings,2.16
484,2024-04-02T13:55:39Z,3d41daf5b30be266bf64bf38ca1da2bb,BetMGM,Los Angeles Clippers,1.71


In [36]:
grouped = df.groupby('Id')
group_sizes = grouped.size()

# Filter groups with less than 6 rows
ids_with_less_than_six_rows = group_sizes[group_sizes < 6].index.tolist()

print("Ids with less than 6 rows:")

ids_with_less_than_six_rows

Ids with less than 6 rows:


['2e3bc94ea7d71c731889bad9d61519da',
 '2e87431523192387ff81ff8d741a0be0',
 '87fb380f7aa6e45faabc77b765e9be26',
 '886bd38845e7ea6f4e176309e542c6be',
 'd5d233f619b7d1aa32f571a77d8f610d',
 'fec1f6e2d2f71182d491f4310335e106']

In [37]:
miss = []
for Id in ids_with_less_than_six_rows:
    group = df[df['Id'] == Id]
    sportsbooks = ["FanDuel", "BetMGM", "DraftKings"]
    
    # check which sportsbook is missing
    present_sportsbooks = group['Sportsbook'].unique()
    missing_sportsbooks = [sb for sb in sportsbooks if sb not in present_sportsbooks][0]

    teams = group['Team'].unique()
    for team in teams:
        missing = [None, None, None, None, float('-inf')]
        # calculate average odds 
        missing[0] = group['Timestamp'].iloc[0]
        missing[1] = Id
        missing[2] = missing_sportsbooks
        missing[3] = team
        missing[4] = round(group[group['Team'] == team]['Odds'].mean(), 2)
        miss.append(missing)
missing_df = pd.DataFrame(miss, columns=['Timestamp', 'Id', 'Sportsbook', 'Team', 'Odds'])

In [38]:
concat = pd.concat([df, missing_df], ignore_index=True)
concat = concat.sort_values(by=['Timestamp', 'Id'])
concat = concat.reset_index(drop=True)
full = concat.merge(concat,
               left_on=['Timestamp','Id', 'Sportsbook'],
               right_on = ['Timestamp','Id', 'Sportsbook'])
full = full.drop(full[full['Team_x'] == full['Team_y']].index)
full = full.reset_index(drop=True)

In [39]:
full

,Timestamp,Id,Sportsbook,Team_x,Odds_x,Team_y,Odds_y
0,2024-03-23T13:55:39Z,0c8f081dd40ab9463ecf78410e799a51,DraftKings,Atlanta Hawks,1.28,Charlotte Hornets,3.80
1,2024-03-23T13:55:39Z,0c8f081dd40ab9463ecf78410e799a51,DraftKings,Charlotte Hornets,3.80,Atlanta Hawks,1.28
2,2024-03-23T13:55:39Z,0c8f081dd40ab9463ecf78410e799a51,FanDuel,Atlanta Hawks,1.27,Charlotte Hornets,4.00
3,2024-03-23T13:55:39Z,0c8f081dd40ab9463ecf78410e799a51,FanDuel,Charlotte Hornets,4.00,Atlanta Hawks,1.27
4,2024-03-23T13:55:39Z,0c8f081dd40ab9463ecf78410e799a51,BetMGM,Atlanta Hawks,1.29,Charlotte Hornets,3.60
...,...,...,...,...,...,...,...
493,2024-04-02T13:55:39Z,f6711a4bb0fb8415631489a8a568f818,DraftKings,San Antonio Spurs,10.50,Denver Nuggets,1.06
494,2024-04-02T13:55:39Z,f6711a4bb0fb8415631489a8a568f818,FanDuel,Denver Nuggets,1.07,San Antonio Spurs,9.70
495,2024-04-02T13:55:39Z,f6711a4bb0fb8415631489a8a568f818,FanDuel,San Antonio Spurs,9.70,Denver Nuggets,1.07
496,2024-04-02T13:55:39Z,f6711a4bb0fb8415631489a8a568f818,BetMGM,Denver Nuggets,1.05,San Antonio Spurs,10.50


In [40]:
full['Timestamp'] = pd.to_datetime(full['Timestamp'])

utc_timezone = pytz.utc
est_timezone = pytz.timezone('US/Eastern')

full['Timestamp'] = full['Timestamp'].dt.tz_convert(est_timezone)
full['Timestamp'] = full['Timestamp'].dt.strftime('%Y-%m-%d')

In [41]:
full['Team_x'] = full['Team_x'].map(mapping)
full['Team_y'] = full['Team_y'].map(mapping)

In [42]:
full

,Timestamp,Id,Sportsbook,Team_x,Odds_x,Team_y,Odds_y
0,2024-03-23,0c8f081dd40ab9463ecf78410e799a51,DraftKings,ATL,1.28,CHO,3.80
1,2024-03-23,0c8f081dd40ab9463ecf78410e799a51,DraftKings,CHO,3.80,ATL,1.28
2,2024-03-23,0c8f081dd40ab9463ecf78410e799a51,FanDuel,ATL,1.27,CHO,4.00
3,2024-03-23,0c8f081dd40ab9463ecf78410e799a51,FanDuel,CHO,4.00,ATL,1.27
4,2024-03-23,0c8f081dd40ab9463ecf78410e799a51,BetMGM,ATL,1.29,CHO,3.60
...,...,...,...,...,...,...,...
493,2024-04-02,f6711a4bb0fb8415631489a8a568f818,DraftKings,SAS,10.50,DEN,1.06
494,2024-04-02,f6711a4bb0fb8415631489a8a568f818,FanDuel,DEN,1.07,SAS,9.70
495,2024-04-02,f6711a4bb0fb8415631489a8a568f818,FanDuel,SAS,9.70,DEN,1.07
496,2024-04-02,f6711a4bb0fb8415631489a8a568f818,BetMGM,DEN,1.05,SAS,10.50


In [43]:
fanduel = full[full['Sportsbook'] == 'FanDuel']
columns = fanduel.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'Fanduel_odds_x','Team_y', 'Fanduel_odds_y']
fanduel.columns = columns
fanduel = fanduel.drop('Sportsbook', axis=1)

draftkings = full[full['Sportsbook'] == 'DraftKings']
columns = draftkings.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'Draftkings_odds_x','Team_y', 'Draftkings_odds_y']
draftkings.columns = columns
draftkings = draftkings.drop('Sportsbook', axis=1)

betMGM = full[full['Sportsbook'] == 'BetMGM']
columns = betMGM.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'BetMGM_odds_x','Team_y', 'BetMGM_odds_y']
betMGM.columns = columns
betMGM = betMGM.drop('Sportsbook', axis=1)

In [44]:
merged_fd_dk = pd.merge(fanduel, draftkings, on=['Timestamp', 'Id', 'Team_x', 'Team_y'], how='left')

# Merge the result with betMGM DataFrame
final = pd.merge(merged_fd_dk, betMGM, on=['Timestamp', 'Id', 'Team_x', 'Team_y'], how='left')

In [45]:
columns = final.columns
columns = ['Timestamp', 'Id', 'Teams_x', 'Fanduel_odds_x','Teams_y', 'Fanduel_odds_y', 'Draftkings_odds_x', 'Draftkings_odds_y', 'BetMGM_odds_x', 'BetMGM_odds_y']
final.columns = columns
final = final.drop('Id', axis=1)

order = ['Timestamp', 'Teams_x', 'Fanduel_odds_x','Draftkings_odds_x', 'BetMGM_odds_x', 'Teams_y', 'Fanduel_odds_y','Draftkings_odds_y', 'BetMGM_odds_y']
final = final[order]

In [46]:
final

,Timestamp,Teams_x,Fanduel_odds_x,Draftkings_odds_x,BetMGM_odds_x,Teams_y,Fanduel_odds_y,Draftkings_odds_y,BetMGM_odds_y
0,2024-03-23,ATL,1.27,1.28,1.29,CHO,4.00,3.80,3.60
1,2024-03-23,CHO,4.00,3.80,3.60,ATL,1.27,1.28,1.29
2,2024-03-23,BRK,3.40,3.40,3.40,NYK,1.34,1.34,1.33
3,2024-03-23,NYK,1.34,1.34,1.33,BRK,3.40,3.40,3.40
4,2024-03-23,TOR,2.94,2.90,2.80,WAS,1.43,1.43,1.44
...,...,...,...,...,...,...,...,...,...
161,2024-04-02,MIN,1.33,1.35,1.33,HOU,3.45,3.30,3.30
162,2024-04-02,OKC,1.45,1.49,1.48,PHI,2.80,2.70,2.70
163,2024-04-02,PHI,2.80,2.70,2.70,OKC,1.45,1.49,1.48
164,2024-04-02,DEN,1.07,1.06,1.05,SAS,9.70,10.50,10.50


In [47]:
save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/odds_data/'
file_name = '2024_04_02_onward.csv'
file_path = save_path + file_name
final.to_csv(file_path)

In [48]:
!pwd

/Users/benjamincheng/Documents/GitHub/Sports-Betting/data


In [51]:
save_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/odds_data/"
file_name = "2021_01_20_onward.csv"
file_path = save_path + file_name
whole = pd.read_csv(file_path, index_col=0)

In [53]:
complete = pd.concat([whole, final], axis=0)

In [56]:
save_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/odds_data/"
file_name = "2021_01_20_onward.csv"
file_path = save_path + file_name
complete.to_csv(file_path)